# Natural Language Processing (ML)

In [ ]:
# Импортировать инструменты
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import roc_auc_score

from scipy.sparse import csr_matrix
from scipy.sparse import hstack

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
comments_corpus = pd.read_csv('../toxic.csv', skipinitialspace=True)

# Проверка данных на наличие проблемых проблем
#comments_corpus.isnull().values.any()
#comments_corpus.duplicated().values.any()

rowsums = comments_corpus.iloc[:,2:].sum(axis=1)
all_zero = (rowsums==0)

print(f"Общее количество комментариев в сете: {len(comments_corpus)}.")
print(f"Количество комментариев категории toxic: {comments_corpus['toxic'].sum()}.")
print(f"Количество комментариев категории severe toxic: {comments_corpus['severe_toxic'].sum()}.")
print(f"Количество комментариев категории obscene: {comments_corpus['obscene'].sum()}.")
print(f"Количество комментариев категории threat: {comments_corpus['threat'].sum()}.")
print(f"Количество комментариев категории insult: {comments_corpus['insult'].sum()}.")
print(f"Количество комментариев категории identity hate: {comments_corpus['identity_hate'].sum()}.")
print(f"Общее количество корректных комментариев: {all_zero.sum()}.")

# Организовать новый столбец данных по нетоксичным комментариям из сета
cc_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
comments_corpus['non_toxic'] = comments_corpus[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
cc_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'non_toxic']
comments_corpus = comments_corpus.replace({'non_toxic':{0:1, 1:0}})

sorted_corpus = comments_corpus[cc_columns].sum().sort_values(ascending=False)

# Нарисовать диаграмму
plt.figure(figsize = (8,9))
ax = sns.barplot(x = sorted_corpus.index, y = sorted_corpus.values, palette = 'colorblind', width = 0.9)#, hue = cc_columns)
plt.xticks(rotation=45)
ax.set(xlabel = 'Категории', ylabel = 'Количество комментариев')
ax.set_title('Диаграмма разделения комментариев по категориям')
plt.show()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#!pip install nltk
#nltk.download('all')

# Оставить токены, содержащие символы английского алфавита (НЕ ТОЛЬКО)

def cleaning(data):
    # Привести текст к нижнему регистру
    data = data.lower()
    # Убрать знаки препинания, цифры
    etc = r'[\s+|[,.\"!?@|_:;，\(\)\'/...=\►╟\─\-\•✆\[\]\&#~`→+*%><\\\{\}\|]]'  # Возможно несовпадение при разных условиях
    data = re.sub(etc,'', data)
    data = re.sub(r'(\d+\s\d+)|(\d+)','', data)
    return data

comment_tokenized = []
lemmatizer = WordNetLemmatizer()

# Построчная обработка сета
for text in comments_corpus['comment_text']:
    text = cleaning(text)
    # Токенизировать текст
    text = word_tokenize(text)
    # Убрать стоп-слова
    text = [lemmatizer.lemmatize(word) for word in text if not word in stopwords.words('english') and word.isalpha()]
    comment_tokenized.append(text)

comments_corpus['comment_tokenized'] = comment_tokenized

# Поиск комментария по индексу
search_id = '000103f0d9cfb60f'
comment_id = comments_corpus[comments_corpus['id'] == search_id]
print(f"Количество токенов по индексу {search_id}: {len(comment_id.iloc[0]['comment_tokenized'])}.")
# Можно продолжить как-нибудь

In [ ]:
comments_corpus['Token_comment'] = [' '.join(item) for item in comments_corpus['comment_tokenized']]
X = comments_corpus['Token_comment']
Y = comments_corpus.drop(['id','Token_comment', 'comment_text', 'comment_tokenized'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=21, shuffle=True)

vectorizer = TfidfVectorizer(max_features=3000)
vectorizer.fit(X_train)
vectorizer.fit(X_test)
X_train_transform = vectorizer.transform(X_train)
X_test_transform = vectorizer.transform(X_test)

print(f"Размерность тренировочной выборки после преобразования текста: {X_train_transform.shape}.")


In [ ]:
clf = RidgeClassifier()
clf.fit(X_train_transform, Y_train)
Y_pred = clf.predict(X_test_transform)
ROC_AUC1 = roc_auc_score(Y_test, Y_pred)
print(f"ROC-AUC: {ROC_AUC1}.")

In [ ]:
unique_words = []

for item in text:
    if item not in unique_words:
        unique_words.append(item)
        res = ''

for parameters in unique_words:
    res += ' ' + ' '.join(parameters)

q = set(res)

# Дополнительные признаки для обучения модели

# 1) ИСПОЛЬЗОВАНИЕ ЧРЕЗМЕРНОЙ КАПИТАЛИЗАЦИИ
c = sum(bool(X) for X in comments_corpus['comment_text'])
caps_lock_train = X_train.apply(lambda x: (len(x.split()))/c)
caps_lock_test = X_test.apply(lambda x: (len(x.split()))/c)

# 2) ЕСЛИ ВООБЩЕ ВОТ ТАК!!!!!!!!!!!!!!!!!!!!!!!!
exclamation_point_train = comments_corpus.loc[X_train.index]['comment_text'].apply(lambda x: x.count('!'))
exclamation_point_test = comments_corpus.loc[X_test.index]['comment_text'].apply(lambda x: x.count('!'))

# 3) Наркотики
censored_drugs = set(['opioids', 'heroin', 'barbiturates', 'krokodil', 'dexamphetamine', 'magic mushrooms', 'LSD', 'cocaine'])
censored_drugs_train = comments_corpus.loc[X_train.index]['comment_text'].apply(lambda x:len(set(x.lower().split()).intersection(censored_drugs))/len(set(x.lower().split())))
censored_drugs_test = comments_corpus.loc[X_test.index]['comment_text'].apply(lambda x: len(set(x.lower().split()).intersection(censored_drugs))/len(set(x.lower().split())))

# 4) Уникальные слова

unique_words_train = comments_corpus.loc[X_train.index]['comment_text'].apply(lambda x: len(set(x.lower().split()).intersection(unique_words))/len(set(x.lower().split())))
unique_words_test = comments_corpus.loc[X_test.index]['comment_text'].apply(lambda x: len(set(x.lower().split()).intersection(unique_words))/len(set(x.lower().split())))

# 5) *и*чт*о*мы*тут*пытаемся*скрыть****звездочками

ast_train = comments_corpus.loc[X_train.index]['comment_text'].apply(lambda x: x.count('*'))
ast_test = comments_corpus.loc[X_test.index]['comment_text'].apply(lambda x: x.count('*'))

# Требуется объединение новых признаков с признаками TF-IDF
new_train = csr_matrix(np.array([caps_lock_train, exclamation_point_train, censored_drugs_train, unique_words_train, ast_train]).T)
new_test = csr_matrix(np.array([caps_lock_test, exclamation_point_test, censored_drugs_test, unique_words_test, ast_test]).T)

# Требуется сложить разреженные матрицы по столбцам
X_train_updated= hstack((X_train_transform, new_train))
X_test_updated = hstack((X_test_transform, new_test))

model = RidgeClassifier()
model.fit(X_train_updated, Y_train)
Y_pred = model.predict(X_test_updated)
ROC_AUC2 = roc_auc_score(Y_test, Y_pred)
print(f"ROC-AUC: {ROC_AUC2} при прошлом значении в {ROC_AUC1}.")

if ROC_AUC2 > ROC_AUC1:
    print('Качество классификации улучшилось.')
else:
    print('Качество классификации ухудшилось.')
